In [1]:
import streamlit as st
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from streamlit_folium import st_folium

# Load data
df = pd.read_csv("DashBoardData.csv", parse_dates=["Date"])

# Streamlit UI
st.title("Air Quality Map with Filters")

# Sidebar filters
date_range = st.sidebar.date_input("Select Date Range", [df["Date"].min(), df["Date"].max()])
measure = st.sidebar.selectbox("Select AQI Measure", ["NO2", "CO", "SO2", "O3"])

# Filter data by date range
start_date, end_date = pd.to_datetime(date_range[0]), pd.to_datetime(date_range[1])
df_filtered = df[(df["Date"] >= start_date) & (df["Date"] <= end_date)]

# Create map
m = folium.Map(location=[39.5, -98.35], zoom_start=4)
marker_cluster = MarkerCluster().add_to(m)

# Color map for AQI levels
def get_color(aqi_group):
    return {
        1: "green",
        2: "yellow",
        3: "orange",
        4: "red",
        5: "purple",
        6: "maroon"
    }.get(aqi_group, "gray")

# Choose AQI column based on measure
aqicol = f"{measure}_AQI_Group"

# Plot filtered data
for _, row in df_filtered.iterrows():
    location = [row.get("Latitude", 0), row.get("Longitude", 0)]
    if location == [0, 0]:
        continue  # Skip if location missing
    label = f"{row['City']}, {row['State']}\n{measure} AQI Group: {row[aqicol]}"
    color = get_color(row[aqicol])
    folium.Marker(
        location=location,
        popup=label,
        icon=folium.Icon(color=color)
    ).add_to(marker_cluster)

# Render map in Streamlit
st_folium(m, width=725, height=500)


ModuleNotFoundError: No module named 'folium'